In [132]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
%matplotlib inline

data = pd.read_csv('Welcome_thinkful_data_cleaning.csv', engine='python')
df = pd.DataFrame(data)

#drop nan values
#df = df.dropna()
print(df.head())
print(df.info())
# split first column into 2 columns
#df['PMID'] = df['PMID/PMCID'].str.split(" ").str.get(0)
#df['PMID'] = df['PMID/PMCID'].apply(lambda x: re.findall('\d*', x)[0])
#df['PMCID'] = df['PMID/PMCID'].apply(lambda x: re.findall('[A-Z]{3}\d*', x))
#test = df['PMCID']
#print(test)
#d = df[df.columns.drop('PMID/PMCID')]
#print(df['PMID/PMCID'].sort_values())

              PMID/PMCID Publisher           Journal title  \
0                    NaN       CUP  Psychological Medicine   
1             PMC3679557       ACS       Biomacromolecules   
2  23043264  PMC3506128        ACS              J Med Chem   
3    23438330 PMC3646402       ACS              J Med Chem   
4   23438216 PMC3601604        ACS              J Org Chem   

                                       Article title  \
0  Reduced parahippocampal cortical thickness in ...   
1  Structural characterization of a Model Gram-ne...   
2  Fumaroylamino-4,5-epoxymorphinans and related ...   
3  Orvinols with mixed kappa/mu opioid receptor a...   
4  Regioselective opening of myo-inositol orthoes...   

  COST (£) charged to Wellcome (inc VAT when charged)  
0                                              £0.00   
1                                           £2381.04   
2                                            £642.56   
3                                            £669.64   
4         

Determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

In [6]:
# find the most common journals and total number of articles for each
df['Journal title'].value_counts()

# notice that the capitalization of the titles is effecting the count
df['Journal title'] = df['Journal title'].str.upper()

# Another issue - there is a case where PLOS ONE shows up as PLOSONE and JOURNAL OF BIOLOGICAL CHEMISTRY IS J BIOL CHEMISTRY!
df['Journal title'] = df['Journal title'].str.replace('PLOSONE','PLOS ONE')
df['Journal title'] = df['Journal title'].str.replace('J BIOL CHEMISTRY','JOURNAL OF BIOLOGICAL CHEMISTRY')
print(df['Journal title'].value_counts().head())


PLOS ONE                           199
JOURNAL OF BIOLOGICAL CHEMISTRY     54
NEUROIMAGE                          29
PLOS PATHOGENS                      24
PLOS GENETICS                       24
Name: Journal title, dtype: int64


In [237]:
# find mean, median, and standard deviation for cost column 

#print(df['COST (£) charged to Wellcome (inc VAT when charged)'].value_counts())

# remove the euro sign leading the values and convert to numeric
def get_nums(x): 
    """pull numbers from euro sign"""
    x = x[1:]
    return x
cost = df['COST (£) charged to Wellcome (inc VAT when charged)'].apply(get_nums)
#saw one record come back as 600.25$
cost = pd.to_numeric(cost, errors='ignore')
#print(cost.std())

# remove the euro sign leading the values and convert to numeric take 2
costcol = df['COST (£) charged to Wellcome (inc VAT when charged)'].apply(lambda x: re.findall('\d*?\.\d{2}', x))


#grab numbers from [] and make them floats
costcol = costcol.str.get(0)
costcol = costcol.astype(float)

# check for Nan and outliers - 47 values of 999999 and 7 null (initially)
#print(costcol.value_counts())
#print(costcol.isnull().sum())

# attempt 1: drop the outliers
#still see two values way above the rest: $201,024 and $192,64
#costcol.hist()
#plt.show()
costcol = cost[(costcol<99999)]

# handle the trailing $ since the \d{2} is not grabbing the first 2 digits after the decimal
costcol = costcol.str.strip('$')
costcol = pd.to_numeric(costcol, errors='coerce')
costcol = costcol.dropna()

#print(costcol)
mean = costcol.mean()
median = costcol.median()
mode = costcol.mode()[0]
stddev = costcol.std()

print('Mean: {}\nMedian: {}\nMode: {}\nStandard Deviation: {}'.format(mean,median,mode,stddev))
print(costcol.describe())

Mean: 1825.8423080637374
Median: 1854.0
Mode: 2040.0
Standard Deviation: 811.1612286870626
count     2071.000000
mean      1825.842308
std        811.161229
min          0.000000
25%       1260.580000
50%       1854.000000
75%       2302.930000
max      13200.000000
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: float64
